<a href="https://colab.research.google.com/github/mostume222/inf-test/blob/master/infa_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 0.** If for some reason you encounter issues navigating the folders in the project or having errors you can delete the project folder in the online colab environment by using the following code cell. If you are just starting to use this fresh colab notebook skip this step.

In [ ]:
!cd ~
!pwd
!ls
!rm -r inf-test/

/content
sample_data
rm: cannot remove 'inf-test/': No such file or directory


**Step 1.** Clone github repository from link https://github.com/mostume222/inf-test.git , this will download the source code and also make the correct project structure to tun the compression program.

In [1]:
!git clone https://github.com/mostume222/inf-test.git
%cd inf-test
!git checkout master #go to master branch to have the correct project structure
!ls

Cloning into 'inf-test'...
remote: Enumerating objects: 343, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 343 (delta 22), reused 137 (delta 12), pack-reused 189 (from 1)
Receiving objects: 100% (343/343), 66.30 MiB | 11.36 MiB/s, done.
Resolving deltas: 100% (97/97), done.
/content/inf-test
Updating files: 100% (165/165), done.
Branch 'master' set up to track remote branch 'master' from 'origin'.
Switched to a new branch 'master'
data  env  infa_main.ipynb  output_compression	README.md  sample  sequences  src


**Step 2.** Install the conda environment in /inf-test/env/infa.yml to install dependencies neded to run the compression procedure.

If you would like to install the dependencies on your own, the important ones are:

* cd-hit 4.8.1  
* bowtie2 2.5.4
* cutadapt 3.5
* python 3.6.13
* perl 5.34.0
* pip 21.2.2

In [ ]:
#install condacolab
!ls
!pip install -q condacolab
import condacolab
condacolab.install()
!ls
#create conda envionment in infa.yml file
!conda env create -f env/infa.yml --quiet
!conda env list

data  env  infa_main.ipynb  output_compression	README.md  sample  sequences  src
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...
condacolab_install.log	env		 output_compression  sample	src
data			infa_main.ipynb  README.md	     sequences
Channels:
 - defaults
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: ...working... done
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
# conda environments:
#
base                     /usr/local
infc                     /usr/local/envs/infc



**Step 3.** Run the following cell to create the function to upload your sequences into the compression procedure.

In [2]:
## Import file upload dependencies
from google.colab import files
import os
import shutil

def input():
  destination_dir = '/content/inf-test/input/'
  os.makedirs(destination_dir, exist_ok=True)
  uploaded = files.upload()
  ## move the uploaded file to input directory
  for filename in uploaded.keys():
      shutil.move(filename, os.path.join(destination_dir, filename))
  ## set and return input name
  name_list = filename.split("/")
  input_fasta = name_list[-1]
  return input_fasta

**Step 4**. Upload your fasta file running the following cell and clicking the "Choose Files" button, this file should be in fasta format. (https://www.ncbi.nlm.nih.gov/genbank/fastaformat/) an example of this file can be found in **"inf-test/sample/release_examples/sequences.fasta"**

In [ ]:
### this cell is to create the input directory and reciebe the fasta file
## import files dependences
input_fasta = input()

Saving 1,.comp to 1,.comp


**Step 5**. Upload your classes file, this file must be in a text base format, column wise separated by a "\t" caracter, where the first column is the "accession id" of each sequence in the input fasta file, the second column has to be the "class id" realted to the referd sequence. An example of this file is in **"inf-test/sample/release_examples/classes.dmp"**

In [ ]:
input_classes = input()

Saving classes.dmp to classes.dmp


**Step 6**. Upload your names file, this file has the names for all "class id" raws in the classes file. This file must be in a text file tab ("\t") separated, the first column being "class id", the second column being "class name". This format facilitates the making of the final pangenome file and makes it easier to use. An example of this file can be found in "inf-test/sample/release_examples/names.dmp"

In [ ]:
input_names = input()

Saving names.dmp to names.dmp


**Step 7.** Run the following cell to take the 3 input files and process them into the correct folders. This procedure will generate the necesary requicites to run the compression algorithm correctly.

In [ ]:
## merge the data and put the files in the correct order
#  run the script with conda env
destination_dir = '/content/inf-test/input/'
%cd /content/inf-test/src/
ifld = destination_dir
dfld = "/content/inf-test/data/"
sfld = "/content/inf-test/sequences/"
!source activate infc && ls
!chmod +x merge.pl
!source activate infc && perl merge.pl {ifld}/{input_fasta} {ifld}/{input_classes} {ifld}/{input_names} {sfld}/{input_fasta}  {dfld}/sci_names.dmp {dfld}/acc_variants.list


/content/inf-test/src
compressor  merge.pl  translate_classes.pl  translate_names.pl


**Step 8.** Run the coompression algorithm with the follwing cell.
The resouls of the compression can be found in "inf-test/output_compression/name_of_your_input/"

"report.dmp" is the compression report, with general statistics about the compression procedure.

"your_file.pan" is the pangenome file, this file includes all dispensable and unique pieces for all classes defined in the classes file (second input).

Inside the "pangenome_unique" folder you can find separated fasta files with the ".u" extension. This files contains the unique pieces for each class that take part into the compression procedure.

**Make the headers descripcion of the pangenome file.**

In [3]:
#  run the script with conda env
%cd /content/inf-test/src/compressor/
!source activate infc && ls
!chmod +x run_compress.sh
!source activate infc && ./run_compress.sh 0.94 0.958 200 50 100000 {input_fasta}


/content/inf-test/src/compressor
/bin/bash: line 1: activate: No such file or directory
/bin/bash: line 1: activate: No such file or directory


In [ ]:
#clean
!rm -r /content/inf-test/output_compression/1,.comp/

rm: cannot remove '/content/inf-test/output_compression/1,.comp/': No such file or directory
